# CoDeepNEAT demo
## CS081 project checkpoint demo
### Harsha Uppli, Alan Zhao, Gabriel Meyer-Lee

The following notebook demonstrates using CoDeepNEAT to solve MNIST

In [1]:
from math import pi, floor
from random import random
from codeepneat import codeepneat, config, population, chromosome, genome, visualize
import pickle
import numpy as np
import keras

Using TensorFlow backend.


### Sample problem: MNIST data set

The MNIST data set of handwritten gray scale images of digits 0-9 is a classic computer vision data set and therefore makes for good testing. Conveniently, it's also built into Keras, which our CoDeepNEAT imiplementation is built off of.

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
trainx = np.reshape(x_train,(60000,28,28,1))
testx = np.reshape(x_test, (10000,28,28,1))
trainy = keras.utils.np_utils.to_categorical(y_train, 10)
testy = keras.utils.np_utils.to_categorical(y_test, 10)
data = [trainx, trainy, testx, testy]

### Configuring NEAT

Many of the options and inputs are still handled through the config file. The config file has been shortened considerably as many parameters have been eliminated, although many parameters have also been introduced which could be added.

In [3]:
%%file configMNIST
#--- parameters for the robot experiment ---#
[phenotype]
input_nodes         = 28,28,1
output_nodes        = 10
conv                = True

[genetic]
max_fitness_threshold = 1

# Human reasoning
pop_size              = 10
prob_addconv          = 1
prob_addlayer         = 0.1
prob_addmodule        = 0.05
elitism               = 1

[genotype compatibility]
compatibility_threshold = 3.0
compatibility_change    = 0.0
excess_coeficient       = 1.0
disjoint_coeficient     = 1.0
connection_coeficient   = 0.4

[species]
species_size        = 5
survival_threshold  = 0.2
old_threshold       = 30
youth_threshold     = 10
old_penalty         = 0.2
youth_boost         = 1.2
max_stagnation      = 15

Overwriting configMNIST


### Fitness

For this demonstration we'll be using supervised learning to train the networks produced by CoDeepNEAT on NMIST and then use their accuracy after 10 epochs as their fitness.

In [4]:
def fitness(network, data):
    network.fit(data[0], data[1],  epochs=5)
    loss, acc = network.evaluate(data[2], data[3])
    return acc

### Evolution

Evolution with CoDeepNEAT is slightly different than evolution with NEAT. The main difference is coevolution, where we have two separate populations with a hierarchical relationship evolving together.

In [5]:
def evolve(n):
    config.load('configMNIST')
    # Create 2 separate populations (size is now defined explicitly, but config file can still be used)
    module_pop = population.Population(20, chromosome.ModuleChromo)
    # As the top hierarchical level, the blueprint population needs to be able to see the module population
    blueprint_pop = population.Population(5, chromosome.BlueprintChromo, module_pop)
    # Most of the actual evolving is now handled outside of the population, by CoDeepNEAT
    # Instead of requiring the user to overwrite the evaluation function, CoDeepNEAT evaluates the populations itself,
    # it simply requires a fitness function for the networks it creates passed in as an argument.
    codeepneat.epoch(n, blueprint_pop, module_pop, 10, fitness, data, save_best=True, name='MNIST')
    # It will still stop if fitness surpasses the max_fitness_threshold in config file
    # Plots the evolution of the best/average fitness
    visualize.plot_stats(module_pop.stats, name="MNISTmod_")
    visualize.plot_stats(blueprint_pop.stats, name="MNISTbp_")
    # Visualizes speciation
    #visualize.plot_species(module_pop.species_log, name="NMISTmod_")
    #visualize.plot_species(blueprint_pop.species_log, name="NMISTbp_")

In [6]:
evolve(5)

-----Generation 0--------
Network 0
Epoch 1/5
60000/60000 [==============================] - 6s - loss: 1.4036 - acc: 0.6943     
Epoch 2/5
60000/60000 [==============================] - 4s - loss: 0.4946 - acc: 0.8663     
Epoch 3/5
60000/60000 [==============================] - 4s - loss: 0.3724 - acc: 0.9053     
Epoch 4/5
60000/60000 [==============================] - 4s - loss: 0.3271 - acc: 0.9196     
Epoch 5/5
 9120/10000 [==========================>...] - ETA: 0s
Network 0 Fitness: 0.9251
Network 1
Epoch 1/5
60000/60000 [==============================] - 3s - loss: 10.4515 - acc: 0.3486     
Epoch 2/5
60000/60000 [==============================] - 4s - loss: 10.0599 - acc: 0.3745     
Epoch 3/5
60000/60000 [==============================] - 4s - loss: 9.9906 - acc: 0.3791     
Epoch 4/5
60000/60000 [==============================] - 4s - loss: 9.9744 - acc: 0.3804     
Epoch 5/5
 9472/10000 [===========================>..] - ETA: 0s
Network 1 Fitness: 0.3903
Network 2
Epoch 1/

KeyboardInterrupt: 

In [ ]:
def eval_best(model_file):
    config.load('configMNIST')
    model = keras.models.load_model(model_file)
    visualize.draw_net(model, "_" + model_file)    
    model.fit(data[0], data[1], epochs=5)
    loss, fitness = model.evaluate(data[2], data[3])
    print("fitness", fitness)

In [ ]:
eval_best("MNIST_best_model_0")